In [51]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_experiments.library.tomography import ProcessTomography
from qiskit import QuantumCircuit
from qiskit_ibm_runtime.fake_provider import FakeKyoto
from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.quantum_info import Chi, Choi, Kraus, SuperOp
import numpy as np
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import gen_funcs
import csv
from qiskit.primitives import BackendEstimator
# from post_proc_funcs import compute_intmdt_maps, compute_Nrhp

def parallel_exp_1q2q(qc_ls: list, backend, qubit_ls: list,
                      analysis='default'):
    ''' generates a ParallelExperiment object that contains floor(127/n) QPT
    experiments where n is the number of qubits of the system. The generated
    experiments can only be implemented on 1 or 2-qubit systems

    inputs
    qc_ls: list of quantum circuits that are repeated sequences
    backend
    qubit_ls: list of qubits generated from gen_qubit_ls with repeat=True
    analysis: analysis method provided by qiskit_experiment
    '''
    # TODO: add a max_circuits options to expand the implementation of
    # experiments onto more qubits by submitting multiple runs to ibm.

    exp_ls = []
    exp_ls = []
    
    for i in range(len(qc_ls)):
        curr_qc = qc_ls[i]
        curr_qubits_used = qubit_ls[i]
        curr_exp = ProcessTomography(curr_qc, backend,
                                     physical_qubits=curr_qubits_used,
                                     analysis=analysis)
        exp_ls.append(curr_exp)
    
    parallel_exp = ParallelExperiment(exp_ls,
                                          flatten_results=False)
    return parallel_exp
    

def gen_circ_ls(base_qc: QuantumCircuit, num_partitions: int):

    ''' Define a base circuit and create a list of circuits that are
    repititions of the base circuits on increasing times
    inputs:
        base_qc: the base quantum circuit to be repeated
        num_partitions: the number of Phi-maps to be used for QPT

    returns:
    qc_ls: list of quantum circuits that are repititions of base circuits
    '''

    qc_ls = []
    for i in range(1, num_partitions+1):
        curr_qc = base_qc.repeat(i)
        curr_qc = curr_qc.decompose(reps=1)
        qc_ls.append(curr_qc)
    return qc_ls


def extract_channel(experiment_data, vectorization):

    ''' extracts choi matrix from a parallel experiment data
    inputs:
    experiment_data: the experiment data output from a parallel experiment
    vectorization: QPT in qiskit ouputs choi matrix representations. If vectorization is true, output superoperator form.

    output:
    proc_ls: list of process operators either in SuperOp or Choi representation
    '''

    proc_ls = []
    data_ls = experiment_data.child_data()
    for i in range(len(data_ls)):
        curr_choi = data_ls[i].analysis_results(dataframe=True)['value'].iloc[0]
        if vectorization:
            curr_superop = SuperOp(curr_choi)
            proc_ls.append(curr_superop)
        else:
            proc_ls.append(curr_choi)

    return proc_ls

def writeCsvFile(fname, data, *args, **kwargs):

    """
    @param fname: string, name of file to write
    @param data: list of list of items

    Write data to file
    """
    mycsv = csv.writer(open(fname, 'wb'), *args, **kwargs)
    for row in data:
        mycsv.writerow(row)


def compute_intmdt_maps(superop_ls):

    ''' computes list of intermediate maps from superoperator list
    input: superop_ls: list of vectorized superoperators

    '''
    dim = 2 ** (superop_ls[0].num_qubits * 2)
    intmdt_map_ls = [0 for i in range(len(superop_ls))]
    intmdt_map_ls[0] = superop_ls[0]
    for i in range(1, len(superop_ls)):
        curr_applied_inv = np.identity(dim)
        for j in range(i):
            curr_applied_inv = curr_applied_inv @ np.linalg.pinv(superop_ls[j])
        intmdt_map_ls[i] = superop_ls[i] @ curr_applied_inv
    return intmdt_map_ls


def compute_Nrhp(intmdt_map_ls, base_circ_time=90):
    gt = []
    for map in intmdt_map_ls:
        gt.append((np.linalg.norm(Choi(map), ord=1) - 1) / base_circ_time)
    Nrhp = sum(gt)
    return Nrhp

In [2]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibm_osaka")
# backend = service.least_busy(operational=True, simulator=False)
backend

<IBMBackend('ibm_osaka')>

In [3]:
backend.num_qubits

127

In [4]:
# simulator = AerSimulator.from_backend(FakeKyoto())
# ns_simulator = AerSimulator.from_backend(GenericBackendV2(num_qubits=2))

In [5]:
base_circ = QuantumCircuit(1)
base_circ.id(0)
base_circ.id(0)
base_circ.id(0)
base_circ.draw()

┌───┐┌───┐┌───┐
q: ┤ I ├┤ I ├┤ I ├
   └───┘└───┘└───┘

In [6]:
circ_ls = gen_circ_ls(base_circ, backend.num_qubits)
qubit_ls = gen_funcs.gen_qubit_ls(backend.num_qubits, 1, repeat=False)

In [7]:
len(circ_ls)

127

In [9]:
parallel_qpt = parallel_exp_1q2q(circ_ls, backend, qubit_ls)

In [10]:
parallel_result = parallel_qpt.run(backend=backend)

In [11]:
parallel_result.child_data()[0].analysis_results(dataframe=1)

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
054ea891,state,ProcessTomography,[Q0],"Choi([[ 0.97274774+0.00000000e+00j, -0.0057642...",None,ibm_osaka,None,2.0,"[1.9298691315679835, 0.04682699318571835, 0.02...","[1.9317370509813598, 0.04869491259909465, 0.02...",True,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
a22fdec7,process_fidelity,ProcessTomography,[Q0],0.963319,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
8be035d7,completely_positive,ProcessTomography,[Q0],True,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
23a315d0,trace_preserving,ProcessTomography,[Q0],False,None,ibm_osaka,None,None,None,None,None,None,None,None,0.04674,"(None, None)"


In [15]:
parallel_result.child_data()[1].analysis_results(dataframe=1)

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
b4626cf3,state,ProcessTomography,[Q1],"Choi([[ 0.98925 +0.j , 0.002 -0.009j ...",None,ibm_osaka,None,2.0,"[1.9570348827005022, 0.026853224857869704, 0.0...","[1.9570348827005022, 0.026853224857869704, 0.0...",False,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
10575a49,process_fidelity,ProcessTomography,[Q1],0.975813,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
cbeea93f,completely_positive,ProcessTomography,[Q1],True,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
762409e2,trace_preserving,ProcessTomography,[Q1],False,None,ibm_osaka,None,None,None,None,None,None,None,None,0.023553,"(None, None)"


In [16]:
parallel_result.child_data()[2].analysis_results(dataframe=1)

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
ee95b67a,state,ProcessTomography,[Q2],"Choi([[ 0.97525 +0.j , 0.004 +0.j ...",None,ibm_osaka,None,2.0,"[1.891612425804629, 0.05100686773509826, 0.037...","[1.891612425804629, 0.05100686773509826, 0.037...",False,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
15d207c2,process_fidelity,ProcessTomography,[Q2],0.93775,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
7b801d2b,completely_positive,ProcessTomography,[Q2],True,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
1cd8ee35,trace_preserving,ProcessTomography,[Q2],False,None,ibm_osaka,None,None,None,None,None,None,None,None,0.040774,"(None, None)"


In [18]:
parallel_result.child_data()[60].analysis_results(dataframe=1)

,name,experiment,components,value,quality,backend,run_time,trace,eigvals,raw_eigvals,rescaled_psd,fitter_metadata,conditional_probability,completely_positive,delta,conditional_measurement_index
cd0051ee,state,ProcessTomography,[Q60],"Choi([[ 0.974 +0.j , 0.02825 -0.0345j ...",None,ibm_osaka,None,2.0,"[0.9838066421673373, 0.8493333721237658, 0.146...","[0.9838066421673373, 0.8493333721237658, 0.146...",False,"{'fitter': 'linear_inversion', 'fitter_time': ...",1.0,True,None,None
7ab046e0,process_fidelity,ProcessTomography,[Q60],0.466687,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
8061c5dc,completely_positive,ProcessTomography,[Q60],True,None,ibm_osaka,None,None,None,None,None,None,None,None,None,None
60cda4ee,trace_preserving,ProcessTomography,[Q60],False,None,ibm_osaka,None,None,None,None,None,None,None,None,0.274894,"(None, None)"


In [23]:
superop_ls = extract_channel(parallel_result, vectorization=1)

In [28]:
superop_ls[3]

SuperOp([[ 0.93675 +0.j      ,  0.002875-0.003375j,  0.002875+0.003375j,
           0.043   +0.j      ],
         [-0.01    -0.00325j ,  0.87025 +0.20425j , -0.015   +0.0045j  ,
          -0.00275 -0.0445j  ],
         [-0.01    +0.00325j , -0.015   -0.0045j  ,  0.87025 -0.20425j ,
          -0.00275 +0.0445j  ],
         [ 0.06325 +0.j      , -0.002875+0.003375j, -0.002875-0.003375j,
           0.957   +0.j      ]],
        input_dims=(2,), output_dims=(2,))

In [26]:
with open("output.txt", "w") as f:
  print(superop_ls, file=f)

In [47]:
superop_ls

[SuperOp([[ 0.97274774+0.00000000e+00j, -0.00413494-1.97751145e-03j,
           -0.00413494+1.97751145e-03j,  0.03109815-1.01405264e-19j],
          [-0.00576428-6.99011288e-03j,  0.95575552+7.74725894e-02j,
           -0.00209825+1.70386270e-02j, -0.00464454-1.35773952e-02j],
          [-0.00576428+6.99011288e-03j, -0.00209825-1.70386270e-02j,
            0.95575552-7.74725894e-02j, -0.00464454+1.35773952e-02j],
          [ 0.02713627+2.56073106e-19j,  0.00209407+4.24413466e-03j,
            0.00209407-4.24413466e-03j,  0.96901784+5.13157834e-19j]],
         input_dims=(2,), output_dims=(2,)),
 SuperOp([[ 0.98925 +0.j      ,  0.004625+0.004875j,  0.004625-0.004875j,
            0.017   +0.j      ],
          [ 0.002   +0.009j   ,  0.9655  +0.10075j , -0.00525 -0.008j   ,
            0.00725 -0.01275j ],
          [ 0.002   -0.009j   , -0.00525 +0.008j   ,  0.9655  -0.10075j ,
            0.00725 +0.01275j ],
          [ 0.01075 +0.j      , -0.004625-0.004875j, -0.004625+0.004875j,
   

In [53]:
intmdt_maps = compute_intmdt_maps(superop_ls)

In [54]:
Nrhp = compute_Nrhp(superop_ls)

In [55]:
Nrhp / (Nrhp + 1)

0.28414525052866063

In [57]:
for i in range(0, len(superop_ls), 5):
    D = []
    curr_intmdt_maps = compute_intmdt_maps(superop_ls[0: i])
    curr_Nrhp = compute_Nrhp(superop_ls[0: i])
    D.append(curr_Nrhp)

IndexError: list index out of range